In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random, copy
from Mylib import *

Start loading Mylib...
Mylib imported!


In [3]:
%matplotlib inline

#### Importing Modeling results

In [4]:
path = '200427_0005_20대_보건복지부_발언정제'
# './result/'
flda = gs.models.LdaMulticore.load(f'./result/{path}/ldafit.pickle')
fid2word = gs.models.LdaMulticore.load(f'./result/{path}/ldafit.pickle.id2word')
inst = pd.read_pickle('./result/200427_0005_20대_보건복지부_발언정제/inst.pickle')

#### Topic Cloud

topcld = [{t:float(s)*10**(1/16) for s,t in zip(re.findall("[0-9.]+", j), re.findall("[^+*0-9.\" ]+", j))} 
          for i, j in flda.print_topics(flda.num_topics, 20)]
WC = WordCloud(width=600, height=600, margin = 1, background_color='white', 
               font_path = './etc/BinggraeMelona.ttf', colormap = 'cool')

ntpc = flda.num_topics
ncols = 4
nrows = int(np.ceil(ntpc/ncols))

fig, axes = plt.subplots(nrows, ncols, figsize = (30, 30))
fig.suptitle('Topic-Term Clouds', fontsize = 40)

for i in tqdm(range(nrows)):
    for j in range(ncols):
        loc = i + j + (ncols - 1)*i
        if loc < len(topcld):
            Tcld = WC.generate_from_frequencies(topcld[loc])
            axes[i, j].imshow(Tcld)
            axes[i, j].axis('off')
            axes[i, j].set_title(f"Topic {str(loc + 1)}", {'fontsize':30})
        else:
            break
plt.savefig(f'./result/{path}_TTcloud.png', dpi = 100)
plt.close()

In [5]:
def nrmlzcorp(corpus, thrsh = 100):
    merged_tokens = my_list(corpus).sum()
    nrmcorp, i, stop = [[]], 0, False
    while not stop:
        while len(nrmcorp[-1]) < thrsh and i < len(merged_tokens):
            nrmcorp[-1].append(merged_tokens[i])
            i += 1
            if i == len(merged_tokens):
                stop = True
        nrmcorp.append([])
    return(nrmcorp[:-1])

def nl(string, over = 5):
    sstring = string.split(' ')
    conc = ['']
    for s in sstring:
        if len(conc[-1]) < over:
            conc[-1] += ' ' + s
        else:
            conc.append(s)
    return('\n'.join(conc)[1:])

#### Getting Topics

In [13]:
raw_data = pd.read_pickle('./result/data_20대_보건복지부_발언정제.pkl')
# raw_data = raw_data.loc[[idx for idx in raw_data.index if raw_data.PARTY.loc[idx] != '非의원']]
raw_data.AGENDA = raw_data.AGENDA.apply(lambda x: x[:30])
nrmcorp = nrmlzcorp(raw_data.token)

In [15]:
for i, (전후, 회차) in enumerate(
    [('정권교체전', [343, 351]), ('정권교체후', [352, 364]), ('전체', [343, 364])]
):
    gdata = raw_data.loc[[idx
                      for n in range(회차[0],회차[1]+1)
                      for idx, m in zip(raw_data.MEETING_NAME.index, raw_data.MEETING_NAME)
                      if re.search(str(n), m) is not None]]
    doC =  {'{}'.format(p):nrmlzcorp(gdata.loc[[idx
                                                for idx in gdata.index
                                                if gdata.loc[idx].PARTY == p]].token, thrsh = 50)
                for p in np.unique(gdata.PARTY) if p != '非의원'}
    if i == 0:
        tops = fitdocs(inst, flda, doC, True)
        tops = tops.assign(ba = 전후)
    else:
        tmp = fitdocs(inst, flda, doC, True)
        tops = tops.append(tmp.assign(ba = 전후))
tops = tops[[tops.columns[-1]] + tops.columns[:-1].to_list()]

In [16]:
tops

,ba,Total,국민의당,더불어민주당,무소속,자유한국당
0,정권교체전,"(2, 20.44)","(2, 32.0)","(12, 17.51)","(7, 14.19)","(2, 26.85)"
1,정권교체전,"(12, 13.64)","(13, 14.0)","(2, 17.51)","(12, 11.49)","(14, 10.23)"
2,정권교체전,"(10, 8.74)","(10, 8.0)","(10, 11.84)","(11, 11.49)","(12, 8.7)"
3,정권교체전,"(13, 7.44)","(4, 7.5)","(3, 7.23)","(13, 10.14)","(5, 8.44)"
4,정권교체전,"(14, 6.5)","(12, 6.5)","(13, 6.39)","(10, 8.11)","(9, 6.91)"
0,정권교체후,"(2, 14.13)","(2, 17.6)","(12, 14.12)","(2, 18.71)","(1, 19.39)"
1,정권교체후,"(1, 11.73)","(1, 13.3)","(2, 13.9)","(12, 12.95)","(2, 12.48)"
2,정권교체후,"(12, 10.03)","(13, 12.88)","(16, 10.36)","(9, 12.95)","(16, 11.29)"
3,정권교체후,"(16, 9.84)","(7, 9.44)","(14, 10.25)","(14, 12.23)","(13, 10.49)"
4,정권교체후,"(14, 8.54)","(14, 8.58)","(3, 9.0)","(5, 10.07)","(5, 9.16)"


In [ ]:
tops.to_html('./result/tmp.html', encoding = 'utf-16')

tops[tops.columns[1:]].applymap(lambda x: int(x[1:3].replace(',', ''))).append(
    tops[tops.columns[1:]].applymap(lambda x: float(re.sub('[ ,)]', '', x[3:])))
).to_excel('./result/tmp.xls')

#### Pie Chart

In [102]:
from matplotlib import font_manager, rc
import seaborn as sns

rc('font', **{'family' : ['AppleMyungjo', 'HCR Batang'][0],
              'weight' : 'bold',
              'size'   : 16})

def my_pie(ax, df, ttl):
    sizes, labels = df.비율.values.tolist(), df['선정 주제명'].values.tolist()
    labels = [nl(l, 5) for l in labels]
    sizes.append(round(100 - sum(sizes), 2)), labels.append('기타')
    ax.pie(x = sizes, labels = labels, autopct='%1.1f%%', shadow=True, rotatelabels=True, 
           radius = 1.2, pctdistance = .3, labeldistance = .45, colors = ['whitesmoke', 'bisque'])
    ax.set_title(ttl, {'fontsize':20, 'verticalalignment': 'bottom'})

sorted(font_manager.findSystemFonts(fontpaths=None, fontext='ttf'))

In [38]:
tmp = pd.read_csv('./result/tmp2.csv')
토픽정리 = pd.read_csv('./result/토픽정리.csv').drop(['토픽 내 단어 사후분포, 내림차순'], 1)
구분들, 분류들 = set(tmp.구분), set(tmp.분류)
tmp = tmp.merge(토픽정리).sort_values(['구분', '분류', '비율'], ascending=False)

In [8]:
tmp.tail(2)

,구분,분류,토픽,비율,선정 주제명
24,당선구분,비례대표,14,8.30,사회복지 문제 전반
27,당선구분,비례대표,9,8.16,의료산업 정책 및 법안


In [120]:
i, j = -1, -1
for 구분 in 구분들:
    i += 1
    nrow = i + 1
    for 분류 in 분류들:
        subdf = tmp.loc[tmp.구분 == 구분].loc[tmp.분류 == 분류]
        if not subdf.empty:
            j += 1
    ncol = j + 1
    j = -1    

fig, axes = plt.subplots(ncols = ncol, nrows = nrow, figsize = (ncol*6, nrow*6), sharex = True, sharey = True)
fig.subplots_adjust(
    left = .0,  # the left side of the subplots of the figure
    right = 1.,   # the right side of the subplots of the figure
    bottom = .0,  # the bottom of the subplots of the figure
    top = .98,     # the top of the subplots of the figure
    wspace = 0., # the amount of width reserved for space between subplots,
                  # expressed as a fraction of the average axis width
    hspace = 0.1  # the amount of height reserved for space between subplots,
                  # expressed as a fraction of the average axis height
)
# fig.suptitle('Top Topics', fontsize = 40)

i, j = -1, -1
for 구분 in 구분들:
    i += 1
    for 분류 in 분류들:
        subdf = tmp.loc[tmp.구분 == 구분].loc[tmp.분류 == 분류]
        if not subdf.empty:
            j += 1
            my_pie(axes[i, j], subdf, 구분 + '_' + 분류)
    ncol = j
    nrow = i
    j = -1

plt.savefig(f'./result/piecharts2.png')
plt.close()

#### Extract documentwise topics from samples

In [19]:
주제명선정 = raw_data.copy()
주제명선정 = 주제명선정.assign(cmpl_topics = [[] for t in 주제명선정.token])

주제명선정 = 주제명선정.loc[[i for i, doc in 주제명선정.token.items() if len(doc) > 20 and len(doc) < 50],
                  ['AGENDA', 'CONFER_NUM', 'SEQ_NO', 'SPEAKER', 'PARTY', 'CONTENT', 'token', 'cmpl_topics']]
주제명선정 = 주제명선정.sort_values(['CONFER_NUM', 'SEQ_NO'])

# random.seed(1021)
start = random.sample(range(len(주제명선정)), 1)[0]
# 주제명선정 = 주제명선정.loc[[2037, 2040, 2041, 2043]]
주제명선정 = 주제명선정.iloc[[i for i in range(start, start + 10)]]

for i, doc in 주제명선정.token.items():
    d2b = fid2word.doc2bow(doc)
    dtopics = flda.get_document_topics(d2b)
    dtopics = sorted(dtopics, key = lambda x: x[1], reverse=True)[:3]
    dtopics = [(tup[0] + 1, round(tup[1]*100, 2)) for tup in dtopics]
    주제명선정.at[i, 'cmpl_topics'] = dtopics

# 주제명선정 = 주제명선정.drop(['CONFER_NUM', 'SEQ_NO'], 1)
주제명선정.CONTENT = pd.read_pickle('./result/HWC_noun_token.pkl').CONTENT.loc[[idx for idx in 주제명선정.index]]
주제명선정.to_html('./result/주제명선정.html', encoding='utf-16')